In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
import constants as c
from requests.auth import HTTPBasicAuth
import utils
import requests
import csv

# Gather Commit Data from BusinessLogic

In [ ]:
azure_pat = os.getenv(c.AZURE_TOKEN_ENV_KEY)

In [ ]:
def get_commit_data(commit):
    commit_id = commit["commitId"]
    author = commit["author"]["email"].split("@")[0]
    date = commit["author"]["date"]
    message = commit["comment"]
    url = commit["remoteUrl"]
    adds = commit["changeCounts"]["Add"]
    deletes = commit["changeCounts"]["Delete"]
    changes = commit["changeCounts"]["Edit"]

    return {
        "commit_id": commit_id,
        "author": author,
        "date": date,
        "message": message,
        "url": url,
        "adds": adds,
        "deletes": deletes,
        "changes": changes
    }

In [ ]:
fieldNames = ["commit_id", "author", "date", "message", "url", "adds", "deletes", "changes"]

# Generate file `commit_data.csv`

In [ ]:
if os.path.exists(c.COMMIT_DATA_FILE_NAME):
    os.remove(c.COMMIT_DATA_FILE_NAME)
skip = 0
top = 100
while True:
    params = {
        "searchCriteria.$top": top,
        "searchCriteria.$skip": skip,
    }
    response = requests.get(c.BASE_COMMIT_URL, auth=HTTPBasicAuth('', azure_pat), params=params)
        

    if response.status_code != 200:
        print("Error: ", response.status_code, response.text)

    data = response.json()["value"]
    if not data:
        print("No more commits to process.")
        break
    print(f"Total commits: {len(data)}")    
    print(f"Processing {skip}-{skip+top} commits...")
    with open(c.COMMIT_DATA_FILE_NAME, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldNames)
        if skip == 0: 
            writer.writeheader()
        for commit in data:
            commit_data = get_commit_data(commit)
            writer.writerow(commit_data)    
    skip += top
    print(f"skip: {skip}, top: {top}")